In [1]:
from sklearn.linear_model import LogisticRegression as logit_model
from sklearn.metrics import confusion_matrix
from OptionsBase.OptionsDataFrame import OptionsDataFrame as odframe
import numpy as np

def fake_labels(size):
    return np.array([1 if np.random.rand() >= .5 else 0 for _ in range(size)])

apple_options = odframe('AAPL').fetch_data()['put_low'].as_matrix()
apple_options = apple_options[np.where(apple_options != '<NA>')].astype(np.float32).reshape(-1, 1)
labels = fake_labels(apple_options.shape[0])

lm = logit_model()
lm.fit(apple_options, labels)
preds = lm.predict(apple_options).astype(np.int32)
print(confusion_matrix(labels, preds))

In [43]:
import merge_data as md
from sklearn.linear_model import LogisticRegression
import numpy as np

options_df = md.options_scrape()
options_df = options_df.apply(lambda entry: np.nan if entry is '<NA>' else entry)
options_df = options_df.dropna()
    

#print(indices)
tickers = list(set(options_df['underlying']))[:1]
underlying_df = md.underlying_scrape(tickers)
joint = md.join_underlying_options(options_df, underlying_df)
labeled_df = md.label_df(joint)

dates = labeled_df['date']
expirations = labeled_df['expdt']
t_deltas = (expirations - dates).map(lambda delta: delta.astype('timedelta64[D]').astype(np.int64))


labeled_df = labeled_df.drop(['date', 'expdt', 'call', 'put', 'underlying'], axis=1)
labeled_df['t_deltas'] = t_deltas

clf = LogisticRegression()
clf = clf.fit(labeled_df.drop('moneyness', axis=1), labeled_df['moneyness'])
print(clf.predict(labeled_df.drop('moneyness', axis=1)))
print(labeled_df['moneyness'].as_matrix())
# nb.plot_roc(labeled_df['moneyness'].as_matrix().reshape(-1, 1), clf.predict(labeled_df.drop('moneyness', axis=1)).reshape(-1, 1))

IBM
[False False False False False False False False False False False False
 False False False False False False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True False False
  True  True  True  True  True  True  True  True False False  True  True
 False False False False False  True  True  True  True  True  True  True
 False False False False False False False False  True  True  True  True
  True False False False False False False False False False False False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True False False False False False False False False False False False
 False False False False False False False Fals

ValueError: too many values to unpack (expected 2)